In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import pandas as pd

#my helper functions from ipynb file
from ipynb.fs.full.PostgreHelper import * 

# Extract

In [11]:
#my sample google sheet
sheet_id = '1xo1_DZEVBiVz39S5Ln3CSnagbPE78cvBDKtwqtC3hKI'

#extract all tables from the sheet
student_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet=Students"
df_student = pd.read_csv(student_url)

st_course_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet=Student_Courses"
df_st_course = pd.read_csv(st_course_url)

course_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet=Courses_Info"
df_course_info = pd.read_csv(course_url)

bank_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet=Bank_Account"
df_bank = pd.read_csv(bank_url)

withdraw_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet=Withdraw_Info"
df_withdraw = pd.read_csv(withdraw_url)

In [13]:
df_st_course

,student_id S001,course_id COURSE001,status In progress,completed date,feedback feedback001
0,S002,COURSE002,Completed,2/1/2022,feedback002
1,S003,COURSE003,In progress,NaN,feedback003
2,S003,COURSE004,In progress,NaN,feedback004
3,S004,COURSE004,Completed,2/3/2022,feedback005
4,S005,COURSE005,In progress,NaN,feedback006
5,S006,COURSE006,Completed,2/5/2022,feedback007
6,S007,COURSE007,In progress,NaN,feedback008
7,S007,COURSE008,In progress,NaN,feedback009
8,S008,COURSE008,Completed,2/7/2022,feedback010
9,S009,COURSE009,In progress,NaN,feedback011


# Transform (Dimensional Model)

# Load

In [ ]:
#connect to my Postgre database that hosted on ElephantSql with free instance
connection, cursor = connect(host = 'arjuna.db.elephantsql.com', 
                             database = '*******', 
                             user = '*******', 
                             password = '*******', 
                             port = '5432')

In [ ]:
#create new table for the above youtube data
query = 'CREATE TABLE youtube_channel \
        (id SERIAL PRIMARY KEY, title text, url text, liked int, disliked int, views int, comment_count int); '
create_table(connection, cursor, query)

In [ ]:
#inserting the dataframe into youtube_channel table

#convert dataframe in tuples to iterate
tuples = [tuple(x) for x in df.to_numpy()]

#df columns into string
cols = ','.join(list(df.columns))

#insert query
query = "INSERT INTO youtube_channel(%s) VALUES(%%s,%%s,%%s,%%s,%%s,%%s)" % (cols)

#inserting with executemany function
try:
    cursor.executemany(query, tuples)
    print("Data inserted using execute_many() successfully!")
except (Exception, psycopg2.DatabaseError) as err:
    print(err)
    cursor.close()

In [ ]:
#read PostgreSQL table into pandas dataframe
data = pd.read_sql_query(con = connection, sql = 'SELECT * FROM youtube_channel')
data.head(10)